# ReadMe

This script calculates the ratio of violent words to non-violent words in articles within a collection.

In [1]:
import pymongo
import pandas as pd

from tqdm import tqdm

In [2]:
def readViolentWords():
    with open("violent_words.txt") as fh:
        target_words = fh.read().split("\n")
    target_words = [x for x in target_words if x not in ["club", "set","burn","war"]]
    target_words.append("aggress") # we want to also have aggressive in there
    return(target_words)

In [3]:
def findMatches(stemmed_list, violent_words):
    matches = [x for x in stemmed_list if x in violent_words]
    return matches

In [4]:
client = pymongo.Connection()
violents = readViolentWords()

In [5]:
# collection = client.politics.articles
# collection = client.politics.debates
collection = client.politics.brown

cursor = collection.find({"stemmed": {"$exists": 1}})
for document in tqdm(cursor, total=cursor.count(), leave=True):
    matches = findMatches(document["stemmed"], violents)
    ratio = len(matches)/len(document["stemmed"])
    
    collection.find_and_modify({
            "_id": document["_id"]
        },{
            "$set": {
                "violent_words": matches,
                "violent_ratio": ratio
            }
        })

|##########| 61/61 100% [elapsed: 00:00 left: 00:00, 212.09 iters/sec]